In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

In [57]:
# Load Chrome driver and movement.uber.com/cities website
PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

# Attributing the city name and the center-most zone code (or origin) to variables so they can be inserted in the URL later
city = 'atlanta' # Note: Atlanta might be missing data from 10/25/2018 - 10/29/2018
origin_code = '1074'
coordinates = '&lat.=33.7489&lng.=-84.4234622&z.=12'

# Open URL for the first day in the desired city (change coordinates depending on city)
driver.get('https://movement.uber.com/explore/' + city + '/travel-times/query?si' + origin_code + '&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=' + 
           '2016-01-02' + '&dt[dr][ed]=' + '2016-01-02' + '&cd=&sa;=&sdn=' + coordinates + '&lang=en-US')

In [58]:
# Defining the necessary funtions for the loop
def choosePrivacy():
    """
    Function that click on the opt out of some privacy settings.
    """
    try:
        priv_pref_buton = driver.find_element_by_id('privacy_pref_optin')
        priv_pref_buton.click()
        
    except:
        pass

    
def pressSkip():
    """
    Function that presses skip on the "Understanding origins" popup window.
    """
    try:
        time.sleep(5)
        skip_button = driver.find_element_by_css_selector('div.af.bk')
        skip_button.click()
            
    except:
        pass


def fillPersonalInfo():
    """"
    Function that fills in the personal information before finally downloading the csv file.
    """
    # name = 'Luiz'
    # last_name = 'Scheuer'
    # email = 'luizgscheuer@gmali.com'
        
    first_name = driver.find_element_by_id('firstName')
    first_name.send_keys('Luiz')

    last_name = driver.find_element_by_id('lastName')
    last_name.send_keys('Scheuer')
                                       
    email = driver.find_element_by_id('email')
    email.send_keys('luizgscheuer@gmali.com')

    purpose_dropdown = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[3]/div/div[2]/div/div/div/form/div[3]/div/div/div/div[1]')
    purpose_dropdown.click()

    purpose = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div/div/div/div/ul/li[6]')
    purpose.click()

    opt_out_newsletter = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[3]/div/div[2]/div/div/div/form/div[5]/div/div/label[2]/div[2]')
    opt_out_newsletter.click()
    

# Agree to privacy preferences
choosePrivacy()
        
    
# Skip button only shows up the first time you open the Chrome browser
pressSkip()

                                    
# Choosing correct data parameters (Traffic Analysis Zone) and opening date bar in preparation for the calendar loop
# Zone type dropdown only shows up the first time you open the Chrome browser
zone_type_dropdown = WebDriverWait(driver, 8).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[3]/div/div[2]/div/div/div/div/div[1]/div[2]/div')))
zone_type_dropdown.click()

traffic_analysis_zones = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div/div/div/ul/li[2]')))
traffic_analysis_zones.click()
        

# Click on Download data
download_button = driver.find_element_by_css_selector('div.f5 button')
download_button.click()


# Download csv file
travel_times_download = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div[3]/div/div[2]/div/div[1]/div/button[1]')
travel_times_download.click()
time.sleep(1)

fillPersonalInfo()

# Download csv file
download_button2 = driver.find_element_by_xpath('//button[contains(@type, "submit")]')
download_button2.click()
time.sleep(2)


In [77]:
# Create URLs
def getURL():
    """"
    Function that creates one URL per date between the specified date range
    """
    date = datetime(2018,8,19)
    while date <= datetime(2020,3,31):
        yield ('https://movement.uber.com/explore/' + city + '/travel-times/query?si' + origin_code + '&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=' +
               date.strftime('%Y-%m-%d') + '&dt[dr][ed]=' + date.strftime('%Y-%m-%d') + '&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US')
        date += timedelta(days=1)


# Perform iteration through URLs downloading the datasets for each URL
iterated_URLs = []
i = 0
print('Number of generated URLs: ' + str(len(list(getURL()))))
for url in getURL():
    i += 1
    driver.execute_script("window.open('"+url+"', '_self')")
    iterated_URLs.append(url)
    print(url)
        
        
    # Agree to privacy preferences
    # optOut()
        
    # Press skip
    # pressSkip()
    
    zone_type_dropdown
    traffic_analysis_zones
    time.sleep(2)
        
    # Click on "Download data"
    download_button = driver.find_element_by_css_selector('div.f5 button') # //button[contains(@data-baseweb, "button")]')
    download_button.click()
    time.sleep(2)
    
    # Download csv file
    travel_times_download = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div[3]/div/div[2]/div/div[1]/div/button[1]')))
    # travel_times_download = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div[3]/div/div[2]/div/div[1]/div/button[1]')
    travel_times_download.click()
    time.sleep(2)

Number of generated URLs: 591
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-08-19&dt[dr][ed]=2018-08-19&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-08-20&dt[dr][ed]=2018-08-20&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-08-21&dt[dr][ed]=2018-08-21&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-08-22&dt[dr][ed]=2018-08-22&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-09-25&dt[dr][ed]=2018-09-25&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-09-26&dt[dr][ed]=2018-09-26&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-09-27&dt[dr][ed]=2018-09-27&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-09-28&dt[dr][ed]=2018-09-28&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-11-01&dt[dr][ed]=2018-11-01&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-11-02&dt[dr][ed]=2018-11-02&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-11-03&dt[dr][ed]=2018-11-03&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-11-04&dt[dr][ed]=2018-11-04&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-12-08&dt[dr][ed]=2018-12-08&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-12-09&dt[dr][ed]=2018-12-09&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-12-10&dt[dr][ed]=2018-12-10&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2018-12-11&dt[dr][ed]=2018-12-11&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-01-14&dt[dr][ed]=2019-01-14&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-01-15&dt[dr][ed]=2019-01-15&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-01-16&dt[dr][ed]=2019-01-16&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-01-17&dt[dr][ed]=2019-01-17&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

...

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-09-30&dt[dr][ed]=2019-09-30&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-10-01&dt[dr][ed]=2019-10-01&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-10-02&dt[dr][ed]=2019-10-02&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-10-03&dt[dr][ed]=2019-10-03&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-11-06&dt[dr][ed]=2019-11-06&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-11-07&dt[dr][ed]=2019-11-07&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-11-08&dt[dr][ed]=2019-11-08&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-11-09&dt[dr][ed]=2019-11-09&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-12-13&dt[dr][ed]=2019-12-13&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-12-14&dt[dr][ed]=2019-12-14&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-12-15&dt[dr][ed]=2019-12-15&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2019-12-16&dt[dr][ed]=2019-12-16&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-01-19&dt[dr][ed]=2020-01-19&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-01-20&dt[dr][ed]=2020-01-20&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-01-21&dt[dr][ed]=2020-01-21&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-01-22&dt[dr][ed]=2020-01-22&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-02-25&dt[dr][ed]=2020-02-25&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-02-26&dt[dr][ed]=2020-02-26&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-02-27&dt[dr][ed]=2020-02-27&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2020-02-28&dt[dr][ed]=2020-02-28&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5

In [ ]:
# Note: could make a list with all desred cities and for city in list, start with search and go back to main page